# CNN - ANN - Intel Image Classification

### Import Libraies

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy  as np
from tensorflow.keras.preprocessing import image

In [3]:
print(tf.__version__)

2.18.0


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory("./seg_train/seg_train",
                                                 target_size=(150,150),
                                                 batch_size=32,
                                                 class_mode='categorical')


Found 14034 images belonging to 6 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = train_datagen.flow_from_directory("./seg_test/seg_test",
                                                 target_size=(150,150),
                                                 batch_size=32,
                                                 class_mode='categorical')


Found 3000 images belonging to 6 classes.


## Part 2 - Building the CNN

### Initialising CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

### Add Convolutional Layer 1


In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[150,150,3]))

c:\Users\Malek\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

### Add Pooling Layer 1

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Add Convolutional Layer 2

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

### Add Pooling Layer 2


In [10]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=6, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
np.set_printoptions(threshold=np.inf,linewidth=np.inf)


In [16]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

c:\Users\Malek\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 179s 402ms/step - accuracy: 0.5540 - loss: 1.2034 - val_accuracy: 0.7083 - val_loss: 0.7881
Epoch 2/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 175s 399ms/step - accuracy: 0.7192 - loss: 0.7443 - val_accuracy: 0.7247 - val_loss: 0.7698
Epoch 3/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 163s 372ms/step - accuracy: 0.7557 - loss: 0.6731 - val_accuracy: 0.7497 - val_loss: 0.6818
Epoch 4/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 164s 373ms/step - accuracy: 0.7828 - loss: 0.5981 - val_accuracy: 0.7803 - val_loss: 0.6100
Epoch 5/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 164s 374ms/step - accuracy: 0.8038 - loss: 0.5387 - val_accuracy: 0.8077 - val_loss: 0.5267
Epoch 6/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 164s 374ms/step - accuracy: 0.8254 - loss: 0.4875 - val_accuracy: 0.8127 - val_loss: 0.5481
Epoch 7/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 163s 372ms/step - accuracy: 0.8333 - loss: 0.4567 - val_accuracy: 0.8170 - val_loss: 0.5304
Epoch 8/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 165s 376ms/step - accuracy: 0.8484 -

## Part 4 - Making prediction

In [17]:
import os

image_folder = './seg_pred/seg_pred'

for img_name in os.listdir(image_folder):
    if img_name.endswith('.jpg'):
        img_path = os.path.join(image_folder, img_name)
        img = image.load_img(img_path, target_size=(150, 150))
        img = image.img_to_array(img) / 255.0
        img = np.expand_dims(img, axis=0)

        prediction = cnn.predict(img)
        predicted_class = list(training_set.class_indices.keys())[np.argmax(prediction)]

        print(f'Image: {img_name} | Predicted Class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Image: 10004.jpg | Predicted Class: street
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: 10005.jpg | Predicted Class: mountain
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image: 10012.jpg | Predicted Class: street
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Image: 10013.jpg | Predicted Class: mountain
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image: 10017.jpg | Predicted Class: mountain
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Image: 10021.jpg | Predicted Class: forest
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Image: 1003.jpg | Predicted Class: sea
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image: 10034.jpg | Predicted Class: glacier
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Image: 10038.jpg | Predicted Class: sea
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image: 10040.jpg | Predicted Class: street
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Image: 10043.jpg | Predicted Class: sea
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Image: 10045.jpg | Predicted Class: street
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39m